##### [warning! If you run inside binder, do not forget to download. Your session will end in ~1h]

# This tutorial is will bring you through your first deep reinforcement learning model


* Seaquest game as an example
* Training a simple lasagne neural network for Q_learning objective


## About OpenAI Gym

* Its a recently published platform that basicly allows you to train agents in a wide variety of environments with near-identical interface.
* This is twice as awesome since now we don't need to write a new wrapper for every game
* Go check it out!
  * Blog post - https://openai.com/blog/openai-gym-beta/
  * Github - https://github.com/openai/gym


## New to Lasagne and AgentNet?
* We only require surface level knowledge of theano and lasagne, so you can just learn them as you go.
* Alternatively, you can find Lasagne tutorials here:
 * Official mnist example: http://lasagne.readthedocs.io/en/latest/user/tutorial.html
 * From scratch: https://github.com/ddtm/dl-course/tree/master/Seminar4
 * From theano: https://github.com/craffel/Lasagne-tutorial/blob/master/examples/tutorial.ipynb
* This is pretty much the basic tutorial for AgentNet, so it's okay not to know it.


# Experiment setup
* Here we basically just load the game and check that it works

In [ ]:
#global params.

#game title. full list of games = http://yavar.naddaf.name/ale/list_of_current_games.html
GAME="space_invaders"

#game image will be resized from (210,160) to your image_size. 
#You may want a bigger image for your homework assignment IF you want a larger NN
IMAGE_W,IMAGE_H = IMAGE_SIZE =(105,80)

#number of parallel agents and batch sequence length (frames)
#For large scale training you may need more agents or experience replay for stability
N_AGENTS = 10
SEQ_LENGTH = 5

In [ ]:
from __future__ import print_function 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%env THEANO_FLAGS="floatX=float32"

In [ ]:
from env import Atari

#creating a game
atari = Atari(GAME,image_size=IMAGE_SIZE) 

action_names = np.array(atari.get_action_meanings())

obs = atari.step(0)[0]

plt.imshow(obs,interpolation='none')

# Basic agent setup
Here we define a simple agent that maps game images into Qvalues using simple convolutional neural network.

![scheme](https://s18.postimg.org/gbmsq6gmx/dqn_scheme.png)

In [ ]:
import lasagne

In [ ]:
from lasagne.layers import InputLayer, DimshuffleLayer

#image observation at current tick goes here, shape = (sample_i,x,y,color)
observation_layer = InputLayer((None,IMAGE_W,IMAGE_H,3))

#reshape to [sample, color, x, y] to allow for convolutional layers to work correctly
observation_reshape = DimshuffleLayer(observation_layer,(0,3,1,2))




In [ ]:
from lasagne.layers import Conv2DLayer,Pool2DLayer,DenseLayer,batch_norm,dropout

#main neural network body
#e.g. 
#Conv2DLayer(<prev layer>,n_filters,filter_size=(8,8),stride=(4,4),name='name')
#DenseLayer(<prev layer,n units,nonlinearity=lasagne.nonlinearities.tanh)

<build a DQN>

#please set this to your last layer for convenience
last_layer = <your last layer>

In [ ]:
#a layer that predicts Qvalues for all actions.
# Just adense layer with corresponding number of units and no nonlinearity (lasagne.nonlinearity.linear)
n_actions = atari.action_space.n
qvalues_layer = <layer that predicts Qvalues>

#To pick actions, we use an epsilon-greedy resolver (epsilon is a property)
from agentnet.resolver import EpsilonGreedyResolver
action_layer = EpsilonGreedyResolver(qvalues_layer,name="e-greedy action picker")

action_layer.epsilon.set_value(np.float32(0.1))


##### Finally, agent
We declare that this network is and MDP agent with such and such inputs, states and outputs

In [ ]:
from agentnet.agent import Agent
#all together
agent = Agent(observation_layers=observation_layer,
              policy_estimators=qvalues_layer,
              action_layers=action_layer)


In [ ]:
#Since it's a single lasagne network, one can get it's weights, output, etc
weights = lasagne.layers.get_all_params(action_layer,trainable=True)
weights

# Create and manage a pool of atari sessions to play with

* To make training more stable, we shall have an entire batch of game sessions each happening independent of others
* Why several parallel agents help training: http://arxiv.org/pdf/1602.01783v1.pdf
* Alternative approach: store more sessions: https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf

In [ ]:
from pool import AtariGamePool

pool = AtariGamePool(agent,GAME, N_AGENTS,image_size=IMAGE_SIZE)


In [ ]:
%%time
#interact for 7 ticks
_,action_log,reward_log,_,_,_  = pool.interact(7)


print(action_names[action_log])
print(reward_log)

In [ ]:
#load first sessions (this function calls interact and remembers sessions)
pool.update(SEQ_LENGTH)

# Q-learning
* An agent has a method that produces symbolic environment interaction sessions
* Such sessions are in sequences of observations, agent memory, actions, q-values,etc
  * one has to pre-define maximum session length.

* SessionPool also stores rewards (Q-learning objective)

In [ ]:
#get agent's Qvalues obtained via experience replay.

#This is an "environment" that replays all stored sessions.
replay = pool.experience_replay

#To only sample several random sessions, try
#replay = pool.experience_replay.sample_session_batch(100,replace=True)

_,_,_,_,qvalues_seq = agent.get_sessions(
    replay,
    session_length=SEQ_LENGTH,
    optimize_experience_replay=True,
)


In [ ]:
#get reference Qvalues according to Qlearning algorithm
from agentnet.learning import qlearning

#loss for Qlearning = (Q(s,a) - (r+gamma*Q(s',a_max)))^2

elwise_mse_loss = qlearning.get_elementwise_objective(qvalues_seq,
                                                      replay.actions,
                                                      replay.rewards,
                                                      replay.is_alive,
                                                      gamma_or_gammas=0.99,)

#compute mean over "alive" fragments
loss = elwise_mse_loss.sum() / replay.is_alive.sum()

In [ ]:
# Compute weight updates. Replace with any optimizer you want
updates = lasagne.updates.adam(loss,weights)

In [ ]:
#compile train function
import theano
train_step = theano.function([],loss,updates=updates)

# Demo run

In [ ]:
untrained_reward = pool.evaluate(save_path="./records",record_video=True)

In [ ]:
from IPython.display import HTML

video_path = <choose video name from what previous cell says>

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_path))


# Training loop

In [ ]:
def get_epsilon(epoch_counter):
    """
    a function which outputs current epsilon for e-greedy exploration given training iteration.
    """
    <implement me!>
    return 0.1

In [ ]:
#starting epoch
epoch_counter = 1

#full game rewards
rewards = {epoch_counter:untrained_reward}

In [ ]:

#the loop may take eons to finish.
#consider interrupting early.
for i in xrange(10**10):    
    
    
    #train
    pool.update(SEQ_LENGTH)
    loss = train_step()
    
    ##update resolver's epsilon (chance of random action instead of optimal one)
    current_epsilon = get_epsilon(epoch_counter)
    action_layer.epsilon.set_value(np.float32(current_epsilon))
    
    if epoch_counter%10==0:
        print("iter=%i\tepsilon=%.3f"%(epoch_counter,current_epsilon))
    
    ##record current learning progress and show learning curves
    if epoch_counter%100 ==0:
        rewards[epoch_counter] = pool.evaluate(record_video=False)
        
        plt.title("random frames")
        for i in range(min((len(pool.games),6))):
            plt.subplot(2,3,i+1)
            plt.imshow(pool.games[i].get_observation())
        plt.show()

    
    epoch_counter  +=1

    
# Time to drink some coffee!

# Evaluating results
 * Here we plot learning curves and sample testimonials

In [ ]:
plt.plot(*zip(*sorted(list(rewards.items()),key=lambda p:p[0])))
    

In [ ]:
rw = pool.evaluate(n_games=20,save_path="./records",record_video=True)
print "mean session score=%f.5"%rw

In [ ]:
from IPython.display import HTML

#select the one you want
video_path= <choose video name from what previous cell says>

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_path))


# Part II. Pre-trained net.

To speed up training, consider using the pre-trained network body and only fine-tuning the qValues layers.

Below you can find a function that loads such pre-trained network. You can use it to initialize the Q-network body.

It may make sense to not train the pre-trained part of body for a few iterations.


In [ ]:
from agentnet.utils.persistence import save,load
def build_pretrained_spaceinvaders(observation_reshape):
    """ a smaller version of DQN pre-trained for ~2 hours on GPU """
    assert tuple(observation_reshape.output_shape) == (None, 3, 105, 80)
    
    #main neural network body
    conv0 = Conv2DLayer(observation_reshape,16,filter_size=(8,8),stride=(4,4),name='conv0')
    conv1 = Conv2DLayer(conv0,32,filter_size=(4,4),stride=(2,2),name='conv1')
    dense0 = DenseLayer(conv1,256,name='dense',nonlinearity=lasagne.nonlinearities.tanh)
    
    load(dense0,"./spaceinvaders_dqn_body_small.pcl")
    return dense0

# Once you got it working,
Try building a network that maximizes the final score

* Moar lasagne stuff: convolutional layers, batch normalization, nonlinearities and so on
* Recurrent agent memory layers, GRUMemoryLayer, etc
* Different reinforcement learning algorithm (p.e. qlearning_n_step), other parameters
* Experience replay pool


Look for info?
* [lasagne doc](http://lasagne.readthedocs.io/en/latest/)
* [agentnet doc](http://agentnet.readthedocs.io/en/latest/)
* [gym homepage](http://gym.openai.com/)


You can also try to expand to a different game: 
 * all OpenAI Atari games are already compatible, you only need to change GAME_TITLE
 * Other discrete action space environments are also accessible this way
 * For continuous action spaces, either discretize actions or use continuous RL algorithms (e.g. .learning.dpg_n_step)
 * Adapting to a custom non-OpenAI environment can be done with a simple wrapper
 
 
__Good luck!__